## ```Imports```
---

In [2]:
import pandas as pd
import numpy as np

## ```Cleaning Data```
---

In [2]:
fraud = pd.read_csv('../capstone_data/fraudTrain.csv')

In [3]:
fraud.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


In [4]:
fraud.shape

(1296675, 23)

In [5]:
fraud['is_fraud'].value_counts()

0    1289169
1       7506
Name: is_fraud, dtype: int64

In [6]:
fraud.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 23 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Unnamed: 0             1296675 non-null  int64  
 1   trans_date_trans_time  1296675 non-null  object 
 2   cc_num                 1296675 non-null  int64  
 3   merchant               1296675 non-null  object 
 4   category               1296675 non-null  object 
 5   amt                    1296675 non-null  float64
 6   first                  1296675 non-null  object 
 7   last                   1296675 non-null  object 
 8   gender                 1296675 non-null  object 
 9   street                 1296675 non-null  object 
 10  city                   1296675 non-null  object 
 11  state                  1296675 non-null  object 
 12  zip                    1296675 non-null  int64  
 13  lat                    1296675 non-null  float64
 14  long              

In [7]:
fraud['is_fraud'].value_counts()

0    1289169
1       7506
Name: is_fraud, dtype: int64

In [8]:
# drop uneccesary columns
fraud.drop(columns=['Unnamed: 0','first','last','street','unix_time','trans_num'], inplace=True)

# change trans_data_trans_time and dob to datetime 
fraud['trans_date_trans_time'] = pd.to_datetime(fraud['trans_date_trans_time'])
fraud['dob'] = pd.to_datetime(fraud['dob'])

# creating hour column to for eda
fraud['hour'] = fraud['trans_date_trans_time'].dt.hour

# creating smaller df with balanced classes for initial modeling/eda
df = fraud[fraud['is_fraud']==0]
df1 = fraud[fraud['is_fraud']==1]
df = df.sample(n=7506, replace=False)
df = pd.concat([df,df1])

In [10]:
# saving updated df to csv
df.sort_index()
df.to_csv('../capstone-dsir82/SmallBalancedClasses.csv', index=False)

## ```Creating Smote Balanced Classes Data Set for final Modeling```
---

In [8]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('../capstone_data/fraudTrain.csv')

In [4]:
df.shape

(1296675, 23)

In [5]:
df.isnull().sum()

Unnamed: 0               0
trans_date_trans_time    0
cc_num                   0
merchant                 0
category                 0
amt                      0
first                    0
last                     0
gender                   0
street                   0
city                     0
state                    0
zip                      0
lat                      0
long                     0
city_pop                 0
job                      0
dob                      0
trans_num                0
unix_time                0
merch_lat                0
merch_long               0
is_fraud                 0
dtype: int64

In [6]:
df['is_fraud'].value_counts()

0    1289169
1       7506
Name: is_fraud, dtype: int64

In [11]:
# Instantiate Smote Class
smo = SMOTE(random_state=42)

#set up X/y
X = df.drop(columns='is_fraud')
y = df['is_fraud']
print(f"Target is_fraud value_counts: {y.value_counts()}")

print('====')
# Synthetically enhance minority class
X_smo, y_smo = smo.fit_resample(X,y)
print(f"Smote Target Prisdum value_counts: {y_smo.value_counts()}")

Target is_fraud value_counts: 0    1289169
1       7506
Name: is_fraud, dtype: int64
====


ValueError: could not convert string to float: '2019-01-01 00:00:18'